In [1]:
from magma import *
from magma.bitutils import int2seq
from loam.boards.icestick import IceStick
from mantle import *

import mantle lattice ice40
import mantle lattice mantle40


<img src="images/counter_diagram.jpg",width=500,height=500>

In [2]:
def DefineCounter(n, has_ce=False):

    name = "Counter{}{}".format(n, has_ce)

    args = ["O", Out(Bits(n)), "COUT", Out(Bit)] + ClockInterface(has_ce=has_ce)
    Counter = DefineCircuit(name, *args)

    add = Adders(n, cin=False, cout=True)
    reg = Register(n, has_ce=has_ce)

    wire( reg.O, add.I0 )
    wire( int2seq(1, n), add.I1 )

    wire(add.O, reg.I)

    wire( reg.O, Counter.O )

    wire( add.COUT, Counter.COUT )

    wireclock(Counter, reg)

    EndCircuit()
    return Counter


In [3]:
from magma.backend.verilog import compile as compile_verilog
print(compile_verilog(DefineCounter(4)))

compiling FullAdder
compiling AdderCout4
compiling Register4
compiling Counter4False
module FullAdder (input  I0, input  I1, input  CIN, output  O, output  COUT);
wire  inst0_O;
wire  inst1_CO;
SB_LUT4 #(.LUT_INIT(16'h9696)) inst0 (.I0(I0), .I1(I1), .I2(CIN), .I3(1'b0), .O(inst0_O));
SB_CARRY inst1 (.I0(I0), .I1(I1), .CI(CIN), .CO(inst1_CO));
assign O = inst0_O;
assign COUT = inst1_CO;
endmodule

module AdderCout4 (input [3:0] I0, input [3:0] I1, output [3:0] O, output  COUT);
wire  inst0_O;
wire  inst0_COUT;
wire  inst1_O;
wire  inst1_COUT;
wire  inst2_O;
wire  inst2_COUT;
wire  inst3_O;
wire  inst3_COUT;
FullAdder inst0 (.I0(I0[0]), .I1(I1[0]), .CIN(1'b0), .O(inst0_O), .COUT(inst0_COUT));
FullAdder inst1 (.I0(I0[1]), .I1(I1[1]), .CIN(inst0_COUT), .O(inst1_O), .COUT(inst1_COUT));
FullAdder inst2 (.I0(I0[2]), .I1(I1[2]), .CIN(inst1_COUT), .O(inst2_O), .COUT(inst2_COUT));
FullAdder inst3 (.I0(I0[3]), .I1(I1[3]), .CIN(inst2_COUT), .O(inst3_O), .COUT(inst3_COUT));
assign O = {inst3_O,inst

In [4]:
icestick = IceStick()

icestick.Clock.on()
icestick.D1.on()
icestick.D2.on()
icestick.D3.on()
icestick.D4.on()
icestick.D5.on()

main = icestick.main()

counter4 = DefineCounter(5, has_ce=True)()
counter23 = DefineCounter(23)()
wire(counter23.COUT, counter4.CE)
wire(counter4.O, [main.D1, main.D2, main.D3, main.D4, main.D5])

In [5]:
print(compile_verilog(main))

compiling FullAdder
compiling AdderCout5
compiling Register5CE
compiling Counter5True
compiling AdderCout23
compiling Register23
compiling Counter23False
compiling main
module FullAdder (input  I0, input  I1, input  CIN, output  O, output  COUT);
wire  inst0_O;
wire  inst1_CO;
SB_LUT4 #(.LUT_INIT(16'h9696)) inst0 (.I0(I0), .I1(I1), .I2(CIN), .I3(1'b0), .O(inst0_O));
SB_CARRY inst1 (.I0(I0), .I1(I1), .CI(CIN), .CO(inst1_CO));
assign O = inst0_O;
assign COUT = inst1_CO;
endmodule

module AdderCout5 (input [4:0] I0, input [4:0] I1, output [4:0] O, output  COUT);
wire  inst0_O;
wire  inst0_COUT;
wire  inst1_O;
wire  inst1_COUT;
wire  inst2_O;
wire  inst2_COUT;
wire  inst3_O;
wire  inst3_COUT;
wire  inst4_O;
wire  inst4_COUT;
FullAdder inst0 (.I0(I0[0]), .I1(I1[0]), .CIN(1'b0), .O(inst0_O), .COUT(inst0_COUT));
FullAdder inst1 (.I0(I0[1]), .I1(I1[1]), .CIN(inst0_COUT), .O(inst1_O), .COUT(inst1_COUT));
FullAdder inst2 (.I0(I0[2]), .I1(I1[2]), .CIN(inst1_COUT), .O(inst2_O), .COUT(inst2_COUT));

In [6]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif ice_counter.blif' ice_counter.v
arachne-pnr -q -d 1k -o ice_counter.txt -p ice_counter.pcf ice_counter.blif
icepack ice_counter.txt ice_counter.bin

iceprog ice_counter.bin

init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x51 0x73 0x10 0x23 0x00 0x35 0x00 0x35 0x06 0x06 0x15 0x43 0xB6
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.
